# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
A. Hughes  ->  A. Hughes  |  ['A. Hughes']
M. Sharma  ->  Y. M. Sharma  |  ['M. Sharma']
N. Wang  ->  N. Wang  |  ['N. Wang']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']
A. Somigliana  ->  A. Somigliana  |  ['A. Somigliana']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']


Arxiv has 50 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/9 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2506.10063


extracting tarball to tmp_2506.10063...

 done.


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2506.10063/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'authors' from 'tmp_2506.10063/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'affils' from 'tmp_2506.10063/affils.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

extracting tarball to tmp_2506.10742...

 done.


A. Somigliana  ->  A. Somigliana  |  ['A. Somigliana']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure fig_MHD-DW_summary_age-pro_v5.pdf

  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\columnwidth]{fig_MHD-DW_summary_age-pro_v5.pdf
}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


Found 131 bibliographic references in tmp_2506.10742/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.10063-b31b1b.svg)](https://arxiv.org/abs/2506.10063) | **Time-scales of polycyclic aromatic hydrocarbon and dust continuum emission from gas clouds compared to molecular gas cloud lifetimes in PHANGS-JWST galaxies**  |
|| J. Kim, et al. -- incl., <mark>K. Kreckel</mark>, <mark>A. Hughes</mark> |
|*Appeared on*| *2025-06-13*|
|*Comments*| *Accepted for publication in ApJ*|
|**Abstract**|            Recent JWST mid-infrared (mid-IR) images, tracing polycyclic aromatic hydrocarbons (PAHs) and dust continuum emission, provide detailed views of the interstellar medium (ISM) in nearby galaxies. Leveraging PHANGS-JWST Cycle 1 and PHANGS-MUSE data, we measure the PAH and dust continuum emission lifetimes of gas clouds across 17 nearby star-forming galaxies by analyzing the relative spatial distributions of mid-IR (7.7-11.3$\mu$m) and H$\alpha$ emission at various scales. We find that the mid-IR emitting time-scale of gas clouds in galaxy disks (excluding centers) ranges from 10 to 30Myr. After star formation is detected in H$\alpha$, mid-IR emission persists for 3-7Myr during the stellar feedback phase, covering 70-80% of the H$\alpha$ emission. This significant overlap is due to intense radiation from star-forming regions, illuminating the surrounding PAHs and dust grains. In most galaxies, the mid-IR time-scale closely matches the molecular cloud lifetime measured with CO. Although mid-IR emission is complex as influenced by ISM distribution, radiation, and abundances of dust and PAHs, the similarity between the two time-scales suggests that once gas clouds form with compact mid-IR emission, they quickly provide sufficient shielding for stable CO formation. This is likely due to our focus on molecular gas-rich regions of galaxies with near-solar metallicity. Finally, we find that the mid-IR emitting time-scale is longer in galaxies with well-defined HII regions and less structured backgrounds, allowing photons to more efficiently heat the ambient ISM surrounding the HII regions, rather than contributing to diffuse emission. This suggests that the shape of the ISM also influences mid-IR emission.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.10719-b31b1b.svg)](https://arxiv.org/abs/2506.10719) | **The ALMA Survey of Gas Evolution of PROtoplanetary Disks (AGE-PRO): I. Program Overview and Summary of First Results**  |
|| K. Zhang, et al. -- incl., <mark>K. Schwarz</mark> |
|*Appeared on*| *2025-06-13*|
|*Comments*| *Accepted for publication in ApJ for the special issue of AGE-PRO*|
|**Abstract**|            We present the ALMA Survey of Gas Evolution of PROtoplanetary Disks (AGE-PRO), a Large Program of the Atacama Large Millimeter/submillimeter Array (ALMA). AGE-PRO aims to systematically trace the evolution of gas disk mass and size throughout the lifetime of protoplanetary disks. It uses a carefully selected sample of 30 disks around M3-K6 stars in three nearby star-forming regions: Ophiuchus (0.5-1 Myr), Lupus (1-3 Myr), and Upper Sco (2-6 Myr). Assuming the three regions had similar initial conditions and evolutionary paths, we find the median gas disk mass appears to decrease with age. Ophiuchus disks have the highest median gas mass (6 M$_{\rm Jup}$), while the Lupus and Upper Sco disks have significantly lower median masses (0.68 and 0.44 M$_{\rm Jup}$, respectively). Notably, the gas and dust disk masses appear to evolve on different timescales. This is evidenced by the median gas-to-dust mass ratio, which decreases from 122 in the youngest disks ($<$1 Myr) to 46 in Lupus disks, and then increases to 120 in the Upper Sco disks. The median gas disk sizes range between 74-110 au, suggesting that typical gas disks are much smaller than those of well-studied, massive disks. Population synthesis models suggest that magneto-hydrodynamic wind-driven accretion can reproduce median disk properties across all three regions, when assuming compact disks with a declining magnetic field over time. In contrast, turbulent-driven models overestimate gas masses of $>$1 Myr disks by an order of magnitude. Here we discuss the program's motivation, survey design, sample selection, observation and data calibration processes, and highlight the initial results.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.10738-b31b1b.svg)](https://arxiv.org/abs/2506.10738) | **The ALMA Survey of Gas Evolution of PROtoplanetary Disks (AGE-PRO): V. Protoplanetary gas disk masses**  |
|| L. Trapman, et al. -- incl., <mark>K. Schwarz</mark> |
|*Appeared on*| *2025-06-13*|
|*Comments*| *Accepted for publication in ApJ for the special issue of AGE-PRO*|
|**Abstract**|            The evolution of the gas mass of planet-forming disks around young stars is crucial for our understanding of planet formation, yet it has proven hard to constrain observationally, due both to the difficulties of measuring gas masses and the lack of a homogeneous sample. Here we present a large grid of thermochemical models which we use to measure protoplanetary gas disk masses of AGE-PRO, the ALMA survey of Gas Evolution in PROtoplanetary disks. AGE-PRO covers a sample of 30 disks around similar spectral type (M3-K6) stars with ages between 0.1 and 10 Myr. Our approach is to simultaneously fit observations of CO isotopologues and N2H+, a complementary molecule produced when CO freezes out. We find that the median gas mass of the three regions decreases over time, from 7.0(+4.4,-2.6)x10^-3 Msun in Ophiuchus (<1 Myr) to 9.4(+5.4,-3.4)x10^-4 Msun for Lupus (~1-3 Myr) and 6.8(+5.1,-2.8)x10^-4 Msun for Upper Sco (~2-6 Myr), with ~1 dex scatter in gas mass in each region. We note that the gas mass distributions for Lupus and Upper Sco look very similar, which could be due to survivorship bias for the latter. The median bulk CO abundance in the CO emitting layer is found to be a factor ~10 lower than the ISM value but does not significantly change between Lupus and Upper Sco. From Lupus to Upper Sco the median gas-to-dust mass ratio increases by a factor ~3 from ~40 to ~120, suggesting efficient inward pebble drift and/or the formation of planetesimals.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.10742-b31b1b.svg)](https://arxiv.org/abs/2506.10742) | **The ALMA Survey of Gas Evolution of PROtoplanetary Disks (AGE-PRO): VII. Testing accretion mechanisms from disk population synthesis**  |
|| B. Tabone, et al. -- incl., <mark>A. Somigliana</mark>, <mark>K. Schwarz</mark> |
|*Appeared on*| *2025-06-13*|
|*Comments*| *Accepted for publication in ApJ for the special issue of AGE-PRO*|
|**Abstract**|            The architecture of planetary systems depends on the evolution of the disks in which they form. In this work, we develop a population synthesis approach to interpret the AGE-PRO measurements of disk gas mass and size considering two scenarios: turbulence-driven evolution with photoevaporative winds and MHD disk-wind-driven evolution. A systematic method is proposed to constrain the distribution of disk parameters from the disk fractions, accretion rates, disk gas masses, and CO gas sizes. We find that turbulence-driven accretion with initially compact disks ($R_0 \simeq 5-20~$au), low mass-loss rates, and relatively long viscous timescales ($t_{\nu,0} \simeq 0.4-3~$Myr or $\alpha_{SS} \simeq 2-4 \times 10^{-4}$) can reproduce the disk fraction and gas sizes. However, the distribution of apparent disk lifetime defined as the $M_D/\dot{M}_*$ ratio is severely overestimated by turbulence-driven models. On the other hand, MHD wind-driven accretion can reproduce the bulk properties of the disk populations from Ophiuchus to Upper Sco assuming compact disks with an initial magnetization of about $\beta \simeq 10^5$ ($\alpha_{DW} \simeq 0.5-1 \times 10^{-3}$) and a magnetic field that declines with time. More studies are needed to confirm the low masses found by AGE-PRO, notably for compact disks that question turbulence-driven accretion. The constrained synthetic disk populations can now be used for realistic planet population models to interpret the properties of planetary systems on a statistical basis.         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.10122-b31b1b.svg)](https://arxiv.org/abs/2506.10122) | **A novel method to trace the dark matter density profile around supermassive black holes with AGN reverberation mapping**  |
|| <mark>M. Sharma</mark>, G. Herrera, N. Arav, S. Horiuchi |
|*Appeared on*| *2025-06-13*|
|*Comments*| *8 pages, 3 figures*|
|**Abstract**|            We propose a new method to determine the dark matter density profile in the vicinity of distant supermassive black holes (SMBH) using reverberation mapping measurements of active galactic nuclei (AGN). For some AGN, the mapping of multiple emission lines allows us to measure the enclosed mass within different radii from the central SMBH, which allows us to infer or constrain the dark matter density profile on sub-pc scales. We find that for five out of fourteen AGN in our identified sample, the observed enclosed mass grows with radii, hinting evidence for the presence of a dark matter component at the 1-2 $\sigma$ level. For these sources, we find global evidence for a universal dark matter profile with a preferred radial steepness of index $\gamma \sim 1.6$, consistent with the scenario expected for a dark matter spike mildly relaxed by stellar heating processes. For the remaining AGN in our sample, we are able to derive upper limits on the dark matter density profile parameters. Interestingly, the strongest evidence for a dark matter component arises from the farthest object considered, 3C 390. This new method allows us to measure or constrain for the first time the dark matter density in the vicinity of extragalactic SMBHs, which is crucial to our understanding of the dynamics and nature of dark matter.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.10565-b31b1b.svg)](https://arxiv.org/abs/2506.10565) | **Searching for radio pulses from radio-quiet gamma-ray pulsars with FAST**  |
|| S. Wang, et al. -- incl., <mark>N. Wang</mark> |
|*Appeared on*| *2025-06-13*|
|*Comments*| *11 pages, 4 figures, Accepted for publication in ApJ*|
|**Abstract**|            We present periodicity and single-pulse searches at 1250 MHz for 22 radio-quiet gamma-ray pulsars, conducted using the Five-hundred-meter Aperture Spherical Radio Telescope (FAST). For PSR J1813$-$1246, we successfully detected pulsed signals with a spin period of 48.08 ms and a dispersion measure of 209.85 ${\rm pc cm^{-3}}$, consistent with the spin period measured at gamma-ray wavelengths. The estimated flux density is approximately 9 $\mu$Jy. For the remaining 21 sources, no radio emission was detected, with flux density upper limits of several $ \mu$Jy. The capability to detect pulsars with such low flux densities provides the opportunity to determine if and how these faint sources differ from much radio-brighter pulsars.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.10731-b31b1b.svg)](https://arxiv.org/abs/2506.10731) | **The ALMA Survey of Gas Evolution of PROtoplanetary Disks (AGE-PRO): II. Dust and Gas Disk Properties in the Ophiuchus Star-forming Region**  |
|| D. A. Ruiz-Rodriguez, et al. -- incl., <mark>K. Schwarz</mark> |
|*Appeared on*| *2025-06-13*|
|*Comments*| *Accepted for publication in ApJ for the special issue of AGE-PRO*|
|**Abstract**|            The ALMA survey of Gas Evolution in PROtoplanetary disks (AGE-PRO) Large Program aims to trace the evolution of gas disk mass and size throughout the lifetime of protoplanetary disks. This paper presents Band-6 ALMA observations of 10 embedded (Class I and Flat Spectrum) sources in the Ophiuchus molecular cloud, with spectral types ranging from M3 to K6 stars, which serve as the evolutionary starting point in the AGE-PRO sample. While we find 4 nearly edge on disks (>70 deg.), and 3 highly inclined disks (>60 deg.) in our sample, we show that, as a population, embedded disks in Ophiuchus are not significantly contaminated by more evolved, but highly inclined sources. We derived dust disk masses from the Band 6 continuum and estimated gas disk masses from the C18O and C17O lines. The mass estimates from the C17O line are slightly higher, suggesting C18O emission might be partially optically thick. While the 12CO and 13CO lines are severely contaminated by extended emission and self-absorption, the C18O and C17O lines allowed us to trace the radial extent of the gaseous disks. From these measurements, we found that the C18O and C17O fluxes correlate well with each other and with the continuum fluxes. Furthermore, the C18O and C17O lines present a larger radial extension than disk dust sizes by factors ranging from 1.5 to 2.5, as it is found for Class II disks using the radial extension of the 12CO. In addition, we have detected outflows in three disks from 12CO observations.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.10734-b31b1b.svg)](https://arxiv.org/abs/2506.10734) | **The ALMA Survey of Gas Evolution of PROtoplanetary Disks (AGE-PRO): III. Dust and Gas Disk Properties in the Lupus Star-forming Region**  |
|| D. Deng, et al. -- incl., <mark>K. Schwarz</mark> |
|*Appeared on*| *2025-06-13*|
|*Comments*| *Accepted for publication in ApJ for the special issue of AGE-PRO*|
|**Abstract**|            We present Band 6 and Band 7 observations of 10 Lupus disks around M3-K6 stars from the ALMA survey of Gas Evolution in PROtoplanetary disks (AGE-PRO) Large Program. In addition to continuum emission in both bands, our Band 6 setup covers the $\mathrm{{}^{12}CO}$, $\mathrm{{}^{13}CO}$ and $\mathrm{C^{18}O}\,J$=2-1 lines, while our Band 7 setup covers the $\mathrm{N_2H^+}\,J$=3-2 line. All of our sources are detected in $\mathrm{{}^{12}CO}$ and $\mathrm{{}^{13}CO}$, 7 out of 10 are detected in $\mathrm{C^{18}O}$, and 3 are detected in $\mathrm{N_2H^+}$. We find strong correlations between the CO isotopologue line fluxes and the continuum flux densities. With the exception of one disk, we also identify a strong correlation between the $\mathrm{C^{18}O}\,J$=2-1 and $\mathrm{N_2H^+}\,J$=3-2 fluxes, indicating similar CO abundances across this sample. For the two sources with well-resolved continuum and $\mathrm{{}^{12}CO}\,J$=2-1 images, we find that their gas-to-dust size ratio is consistent with the median value of $\sim 2$ inferred from a larger sample of Lupus disks. We derive dust disk masses from continuum flux densities. We estimate gas disk masses by comparing $\mathrm{C^{18}O}\,J$=2-1 line fluxes with those predicted by the limited grid of self-consistent disk models of Ruaud et al. (2022). A comparison of these mass estimates with those derived by Trapman et al. (2025), using a combination of CO isotopologue and $\mathrm{N_2H^+}$ line emission, shows that the masses are consistent with each other. Some discrepancies appear for small and faint disks, but they are still within the uncertainties. Both methods find gas disk masses increase with dust disk masses, and gas-to-dust mass ratios are between $10-100$ in the AGE-PRO Lupus sample.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.10735-b31b1b.svg)](https://arxiv.org/abs/2506.10735) | **The ALMA Survey of Gas Evolution of PROtoplanetary Disks (AGE-PRO): IV. Dust and Gas Disk Properties in the Upper Scorpius Star-forming Region**  |
|| C. Agurto-Gangas, et al. -- incl., <mark>K. Schwarz</mark> |
|*Appeared on*| *2025-06-13*|
|*Comments*| *Accepted for publication in ApJ for the special issue of AGE-PRO*|
|**Abstract**|            The Atacama Large Millimeter/submillimeter Array (ALMA) large program AGE-PRO explores protoplanetary disk evolution by studying gas and dust across various ages. This work focuses on ten evolved disks in Upper Scorpius, observed in dust continuum emission, CO and its isotopologues, and N$_2$H$^+$ with ALMA Bands 6 and 7. Disk radii, from the radial location enclosing 68% of the flux, are comparable to those in the younger Lupus region for both gas and dust tracers. However, solid masses are about an order of magnitude below those in Lupus and Ophiuchus, while the dust spectral index suggests some level of dust evolution. These empirical findings align with a combination of radial drift, dust trapping, and grain growth into larger bodies. A moderate correlation between CO and continuum fluxes suggests a link between gas and dust content, through the increased scatter compared to younger regions, possibly due to age variations, gas-to-dust ratio differences, or CO depletion. Additionally, the correlation between C$^{18}$O and N$_2$H$^+$ fluxes observed in Lupus persists in Upper Sco, indicating a relatively stable CO gas abundance over the Class II stage of disk evolution. In conclusion, the AGE-PRO survey of Upper Scorpius disks reveals intriguing trends in disk evolution. The findings point towards potential gas evolution and the presence of dust traps in these older disks. Future high-resolution observations are needed to confirm these possibilities and further refine our understanding of disk evolution and planet formation in older environments.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2506.10063/./obs_ngc1087.png', 'tmp_2506.10063/./heatmap_clean7.7.png', 'tmp_2506.10063/./heatmap_clean10.png', 'tmp_2506.10063/./heatmap_clean11.3.png', 'tmp_2506.10063/./obs_ngc1385.png']
copying  tmp_2506.10063/./obs_ngc1087.png to _build/html/
copying  tmp_2506.10063/./heatmap_clean7.7.png to _build/html/
copying  tmp_2506.10063/./heatmap_clean10.png to _build/html/
copying  tmp_2506.10063/./heatmap_clean11.3.png to _build/html/
copying  tmp_2506.10063/./obs_ngc1385.png to _build/html/
exported in  _build/html/2506.10063.md
    + _build/html/tmp_2506.10063/./obs_ngc1087.png
    + _build/html/tmp_2506.10063/./heatmap_clean7.7.png
    + _build/html/tmp_2506.10063/./heatmap_clean10.png
    + _build/html/tmp_2506.10063/./heatmap_clean11.3.png
    + _build/html/tmp_2506.10063/./obs_ngc1385.png
found figures ['tmp_2506.10719/figures/kmf_medians.png', 'tmp_2506.10719/figures/Fig_AGEPRO_age_groups.png', 'tmp_2506.10719/figures/kmf_medians_4ages.png']
copying  tmp_2506.1

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\Ox}{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\newcommand{\UGent}{Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}$
$\newcommand{\STScI}{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{\MPIA}{Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117, Heidelberg, Germany}$
$\newcommand{\AURA}{AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{\UCSD}{Department of Astronomy \& Astrophysics, University of California, San Diego, 9500 Gilman Dr., La Jolla, CA 92093, USA}$
$\newcommand{\JHU}{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}$
$\newcommand{\OSU}{Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, OH 43210, USA}$
$\newcommand{ÇAPP}{Center for Cosmology and Astroparticle Physics (CCAPP), 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\newcommand{\ARI}{Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstr. 12-14, D-69120 Heidelberg, Germany}$
$\newcommand{\UConn}{Department of Physics, University of Connecticut, 196A Auditorium Road, Storrs, CT 06269, USA}$
$\newcommand{\UHawaii}{Institute for Astronomy, University of Hawaii, 2680 Woodlawn Drive, Honolulu, HI 96822, USA}$
$\newcommand{\UniCA}{Université C\^{o}te d'Azur, Observatoire de la C\^{o}te d'Azur, CNRS, Laboratoire Lagrange, 06000, Nice, France}$
$\newcommand{\UAlberta}{Dept. of Physics, University of Alberta, 4-183 CCIS, Edmonton, Alberta, T6G 2E1, Canada}$
$\newcommand{\Arcetri}{INAF — Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50125, Florence, Italy}$
$\newcommand{\UWyoming}{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\newcommand{\LJMU}{Astrophysics Research Institute, Liverpool John Moores University, 146 Brownlow Hill, Liverpool L3 5RF, UK}$
$\newcommand{\ITA}{Universität Heidelberg, Zentrum für Astronomie, Institut für Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\newcommand{\CfA}{Center for Astrophysics \mid Harvard \& Smithsonian, 60 Garden St., 02138 Cambridge, MA, USA}$
$\newcommand{\MPE}{Max-Planck-Institut für Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\newcommand{\Surrey}{Department of Physics, University of Surrey, Guildford GU2 7XH, UK}$
$\newcommand{\ESO}{European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei München, Germany}$
$\newcommand{\IWR}{Universität Heidelberg, Interdisziplinäres Zentrum für Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\newcommand{\ulyon}{Univ Lyon, Univ Lyon1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574, F-69230 Saint-Genis-Laval France}$
$\newcommand{\COOL}{Cosmic Origins Of Life (COOL) Research DAO, \href{https://coolresearch.io}{https://coolresearch.io}}$
$\newcommand{\OAN}{Observatorio Astron{ó}mico Nacional (IGN), C/ Alfonso XII 3, E-28014 Madrid, Spain}$
$\newcommand{\UBonn}{Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121 Bonn, Germany}$
$\newcommand{\kipac}{Kavli Institute for Particle Astrophysics \& Cosmology (KIPAC), Stanford University, CA 94305, USA}$
$\newcommand{\Umanc}{Jodrell Bank Centre for Astrophysics, Department of Physics and Astronomy, University of Manchester, Oxford Road, Manchester M13 9PL, UK}$
$\newcommand{\NRAO}{National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903, USA}$
$\newcommand{\ANU}{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\newcommand{\AThreeD}{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}$
$\newcommand{\IAC}{Instituto de Astrofísica de Canarias, C/ Vía Láctea s/n, E-38205, La Laguna, Spain}$
$\newcommand{\ULL}{Departamento de Astrofísica, Universidad de La Laguna, Av. del Astrofísico Francisco Sánchez s/n, E-38206, La Laguna, Spain}$
$\newcommand{\Princeton}{Department of Astrophysical Sciences, Princeton University, 4 Ivy Lane, Princeton, NJ 08544, USA}$
$\newcommand{\IRAM}{IRAM, 300 rue de la Piscine, 38400 Saint Martin d'Héres, France}$
$\newcommand{\LERMA}{LERMA, Observatoire de Paris, PSL Research University, CNRS, Sorbonne Universités, 75014 Paris}$
$\newcommand{\YB}{Centro de Desarrollos Tecnológicos, Observatorio de Yebes (IGN), 19141 Yebes, Guadalajara, Spain}$
$\newcommand{\Whitman}{Whitman College, 345 Boyer Avenue, Walla Walla, WA 99362, USA}$
$\newcommand{\IRAP}{IRAP, OMP, UPS, Université de Toulouse, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}$
$\newcommand{\ESOChile}{European Southern Observatory (ESO), Alonso de Córdova 3107, Casilla 19, Santiago 19001, Chile}$</div>



<div id="title">

# Time-scales of polycyclic aromatic hydrocarbon and dust continuum emission from gas clouds compared to molecular gas cloud lifetimes in PHANGS-JWST galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.10063-b31b1b.svg)](https://arxiv.org/abs/2506.10063)<mark>Appeared on: 2025-06-13</mark> -  _Accepted for publication in ApJ_

</div>
<div id="authors">

J. Kim, et al. -- incl., <mark>K. Kreckel</mark>, <mark>A. Hughes</mark>

</div>
<div id="abstract">

**Abstract:** Recent JWST mid-infrared (mid-IR) images, tracing polycyclic aromatic hydrocarbons (PAHs) and dust continuum emission, provide detailed views of the interstellar medium (ISM) in nearby galaxies. Leveraging PHANGS-JWST Cycle 1 and PHANGS-MUSE data, we measure the PAH and dust continuum emission lifetimes of gas clouds across 17 nearby star-forming galaxies by analyzing the relative spatial distributions of mid-IR (7.7-11.3 $\mu$ m) and H $\alpha$ emission at various scales. We find that the mid-IR emitting time-scale of gas clouds in galaxy disks (excluding centers) ranges from 10 to 30 Myr. After star formation is detected in H $\alpha$ , mid-IR emission persists for 3-7 Myr during the stellar feedback phase, covering 70-80 \% of the H $\alpha$ emission. This significant overlap is due to intense radiation from star-forming regions, illuminating the surrounding PAHs and dust grains. In most galaxies, the mid-IR time-scale closely matches the molecular cloud lifetime measured with CO. Although mid-IR emission is complex as influenced by ISM distribution, radiation, and abundances of dust and PAHs, the similarity between the two time-scales suggests that once gas clouds form with compact mid-IR emission, they quickly provide sufficient shielding for stable CO formation. This is likely due to our focus on molecular gas-rich regions of galaxies with near-solar metallicity. Finally, we find that the mid-IR emitting time-scale is longer in galaxies with well-defined Hii regions and less structured backgrounds, allowing photons to more efficiently heat the ambient ISM surrounding the Hii regions, rather than contributing to diffuse emission. This suggests that the shape of the ISM also influences mid-IR emission.

</div>

<div id="div_fig1">

<img src="tmp_2506.10063/./obs_ngc1087.png" alt="Fig17" width="100%"/>

**Figure 17. -** _Top:_ Composite three color images of NGC 1087 created using CO (blue), H$\alpha$(red), and mid-IR (green) observations. Each panel, from left to right, represents mid-IR emission of F770W, F1000W, and F1130W, respectively. The mid-IR observations have been convolved and regridded to match the coarser resolution and pixel grid of of H$\alpha$ observations (see Section \ref{ssec:post}). For visualization purposes, a power-law brightness scale with gamma correction ($\gamma=2$) has been applied in the top panels. The color bars on the right reflect the true flux ranges in each observation. The left panel highlights the area analyzed, outlined by a polygon. It excludes the crowded galaxy center (ellipse) as well as artifacts and exceptionally bright peaks (circles). _Bottom:_ Locations of identified H$\alpha$(orange triangles), CO (blue filled circles), and mid-IR (green open circles) emission peaks (see Section \ref{sec:method}) are overlaid on CO map, shown in grayscale with a linear brightness scale. Again, from left to right, the mid-IR emission peaks correspond to peaks identified in F770W, F1000W, and F1130W maps respectively. The CO grayscale image uses the same intensity range as the CO emission in the top panels shown in blue, with range of flux indicated in the color bar on the right. (*fig:obs_1087*)

</div>
<div id="div_fig2">

<img src="tmp_2506.10063/./heatmap_clean7.7.png" alt="Fig9.1" width="33%"/><img src="tmp_2506.10063/./heatmap_clean10.png" alt="Fig9.2" width="33%"/><img src="tmp_2506.10063/./heatmap_clean11.3.png" alt="Fig9.3" width="33%"/>

**Figure 9. -** Spearman's rank correlation coefficients measured between galaxy properties (columns; Section \ref{ssec:env}) and our measurements (rows; Section \ref{sec:result}). From (a) to (c), correlations obtained using different mid-IR bands (7.7, 10, and 11.3 $\mu$m) are shown, respectively. Statistically strong correlations are highlighted as black squares, with red indicating a positive correlation and blue indicating a negative one. Our measurements are the mid-IR emitting time-scale ($t_{\rm g}$), feedback time-scale ($t_{\rm fb}$), region separation length ($\lambda$), ratios between time-scales ($t_{\rm g}/t_{\rm g}^{\rm CO}$, $t_{\rm fb}/t_{\rm g}$, $t_{\rm fb}/t_{\rm s}$), duration of the mid-IR emitting phase dark in CO and H$\alpha$($t_{\rm CO-dark}$), difference between the overlap time-scales measured with mid-IR versus CO ($\Delta t_{\rm fb, CO}$), and diffuse emission fractions ($f_{\rm diffuse}^{\rm mid-IR}$ and $f_{\rm diffuse}^{\rm H\alpha}$). We correlate these measurements with global galaxy properties, average kpc-scale galaxy properties, average GMC properties, average surface flux density contrasts between emission peaks and galactic average measured in mid-IR and H$\alpha$ maps, and systematic biases described in Section \ref{ssec:env}.
 (*fig:hm*)

</div>
<div id="div_fig3">

<img src="tmp_2506.10063/./obs_ngc1385.png" alt="Fig20" width="100%"/>

**Figure 20. -** Similar to Figure \ref{fig:obs_1087}, showing observations and identified emission peaks of NGC 1385. (*fig:obs_others*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.10063"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\ce}{C^{18}O}$
$\newcommand{\ct}{^{13}CO}$
$\newcommand{\cseven}{C^{17}O}$
$\newcommand{\kms}{ km s^{-1}}$
$\newcommand{\nco}{N_{\rm CO}~}$
$\newcommand$
$\newcommand{\todo}[1]{\textcolor{red}{\textbf{#1}}}$
$\newcommand{\Coco}[1]{\textcolor{black}{\textbf{#1}}}$</div>



<div id="title">

# The ALMA Survey of Gas Evolution of PROtoplanetary Disks (AGE-PRO): \ I. Program Overview and Summary of First Results

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.10719-b31b1b.svg)](https://arxiv.org/abs/2506.10719)<mark>Appeared on: 2025-06-13</mark> -  _Accepted for publication in ApJ for the special issue of AGE-PRO_

</div>
<div id="authors">

K. Zhang, et al. -- incl., <mark>K. Schwarz</mark>

</div>
<div id="abstract">

**Abstract:** We present the ALMA Survey of Gas Evolution of PROtoplanetary Disks (AGE-PRO), a Large Program of the Atacama Large Millimeter/submillimeter Array (ALMA). AGE-PRO aims to systematically trace the evolution of gas disk mass and size throughout the lifetime of protoplanetary disks. It uses a carefully selected sample of 30 disks around M3-K6 stars in three nearby star-forming regions: Ophiuchus (0.5-1 Myr), Lupus (1-3 Myr), and Upper Sco (2-6 Myr).Assuming the three regions had similar initial conditions and evolutionary paths, we find the median gas disk mass appears to decrease with age. Ophiuchus disks have the highest median gas mass (6 M $_{\rm Jup}$ ), while the Lupus and Upper Sco disks have significantly lower median masses (0.68 and 0.44 M $_{\rm Jup}$ , respectively). Notably, the gas and dust disk masses appear to evolve on different timescales. This is evidenced by the median gas-to-dust mass ratio, which decreases from 122 in the youngest disks ( $<$ 1 Myr) to 46 in Lupus disks, and then increases to 120 in the Upper Sco disks. The median gas disk sizes range between 74-110 au, suggesting that typical gas disks are much smaller than those of well-studied, massive disks. Population synthesis models suggest that magneto-hydrodynamic wind-driven accretion can reproduce median disk properties across all three regions, when assuming compact disks with a declining magnetic field over time. In contrast, turbulent-driven models overestimate gas masses of $>$ 1 Myr disks by an order of magnitude. Here we discuss the program's motivation, survey design, sample selection, observation and data calibration processes, and highlight the initial results.

</div>

<div id="div_fig1">

<img src="tmp_2506.10719/figures/kmf_medians.png" alt="Fig8" width="100%"/>

**Figure 8. -**  Comparison of disk properties in the Ophiuchus, Lupus, and Upper Sco samples. The circles are individual values and the squares are median values listed in Table \ref{tab:median}. The gas disk sizes, R$_{\rm CO, 90\%}$ are measured as radius that encloses 90 \% of the $^{12}$CO (2-1). Due to the severe cloud/envelope contamination in the $^{12}$CO emission of the Ophiuchus disks, we only measure R$_{\rm CO, 90\%}$ for the Lupus and Upper Sco disks  ([Trapman, Vioque and Kurtovic 2025]()) . The CO abundances of Ophiuchus disks are assumed to be at the ISM level  ([Zhang, Schwarz and Bergin 2020](), [Trapman, Zhang and Rosotti 2025]())  rather than constrained by \ntwoh, and therefore, are not plotted. The spectral index of Ophiuchus disks is not constrained due to the lack of Band 7 observations. (*fig:disk_properties*)

</div>
<div id="div_fig2">

<img src="tmp_2506.10719/figures/Fig_AGEPRO_age_groups.png" alt="Fig10" width="100%"/>

**Figure 10. -** Isochronal ages of the Lupus and Upper Sco samples of AGE-PRO. The circles are individual ages and the squares represent the median ages of each group. The ages were derived based on stellar properties listed in Table \ref{table:stars} and PMS evolutionary tracks of [ and Feiden (2016)](), [Baraffe, et. al (2015)](). The individual ages and their uncertainties can be found in [Deng, Vioque and Pascucci (2025)](), [Agurto-Gangas, Pérez and Sierra (2025)]().   (*fig:age_distribution*)

</div>
<div id="div_fig3">

<img src="tmp_2506.10719/figures/kmf_medians_4ages.png" alt="Fig13" width="100%"/>

**Figure 13. -** Comparison of disk properties in the Ophiuchus (0.5-1 Myr), Lupus (1-3 Myr), young Upper Sco group (2-3 Myr), and old Upper Sco group (4-6 Myr). This is the same as Figure \ref{fig:disk_properties}, but with the Upper Sco sample divided into two groups that show different isochronal ages (see Figure \ref{fig:age_distribution}). The circles are individual values and the squares are median values, further details in caption of Figure \ref{fig:disk_properties}.
 (*fig:4group_disk_properties*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.10719"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\dr}[0]{\ensuremath{\mathrm{d}r}}$
$\newcommand{\mdisk}[0]{\ensuremath{M_{\rm disk}}}$
$\newcommand{\mgas}[0]{\ensuremath{M_{\rm gas}}}$
$\newcommand{\mdust}[0]{\ensuremath{M_{\rm dust}}}$
$\newcommand{\rgas}[0]{\ensuremath{R_{\rm CO, 90\%}}}$
$\newcommand{\rdust}[0]{\ensuremath{R_{\rm dust, 90\%}}}$
$\newcommand{\rc}[0]{\ensuremath{R_{\rm c}}}$
$\newcommand{\macc}[0]{\ensuremath{\dot{M}_{\rm acc}}}$
$\newcommand{\mstar}[0]{\ensuremath{M_{*}}}$
$\newcommand{\msun}[0]{\ensuremath{\mathrm{M}_{\odot}}}$
$\newcommand{\nhp}[0]{\ensuremath{\mathrm{N}_2\mathrm{H}^+}}$
$\newcommand{\co}[0]{\ensuremath{^{12}\mathrm{CO}}}$
$\newcommand{\xco}[0]{\ensuremath{^{13}\mathrm{CO}}}$
$\newcommand{\cyo}[0]{\ensuremath{\mathrm{C}^{18}\mathrm{O}}}$
$\newcommand{\zet}[0]{\ensuremath{\zeta_{\rm CR}}}$
$\newcommand{\htwo}[0]{\ensuremath{\mathrm{H}_{2}}}$
$\newcommand{\abu}[0]{\ensuremath{x_{\rm CO}}}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# The ALMA Survey of Gas Evolution of PROtoplanetary Disks (AGE-PRO): V.  Protoplanetary gas disk masses

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.10738-b31b1b.svg)](https://arxiv.org/abs/2506.10738)<mark>Appeared on: 2025-06-13</mark> -  _Accepted for publication in ApJ for the special issue of AGE-PRO_

</div>
<div id="authors">

L. Trapman, et al. -- incl., <mark>K. Schwarz</mark>

</div>
<div id="abstract">

**Abstract:** The evolution of the gas mass of planet-forming disks around young stars is crucial for our understanding of planet formation, yet it has proven hard to constrain observationally, due both to the difficulties of measuring gas masses and the lack of a homogeneous sample.Here we present a large grid of thermochemical models which we use to measure protoplanetary gas disk masses of AGE-PRO, the ALMA survey of Gas Evolution in PROtoplanetary disks. AGE-PRO covers a sample of 30 disks around similar spectral type (M3-K6) stars with ages between 0.1 and 10 Myr.Our approach is to simultaneously fit observations of CO isotopologues and $\nhp$ , a complementary molecule produced when CO freezes out.We find that the median gas mass of the three regions decreases over time, from $7.0^{+4.4}_{-2.6}\times10^{-3} \msun$ in Ophiuchus ( $\lesssim$ 1 Myr) to $9.4^{+5.4}_{-3.4}\times10^{-4} \msun$ for Lupus ( $\sim$ 1-3 Myr) and $6.8^{+5.1}_{-2.8}\times10^{-4} \msun$ for Upper Sco ( $\sim$ 2-6 Myr), with $\sim1$ dex scatter in gas mass in each region. We note that the gas mass distributions for Lupus and Upper Sco look very similar, which could be due to survivorship bias for the latter.The median bulk CO abundance in the CO emitting layer is found to be a factor $\sim10$ lower than the ISM value but does not significantly change between Lupus and Upper Sco.From Lupus to Upper Sco the median gas-to-dust mass ratio increases by a factor $\sim3$ from $\sim40$ to $\sim120$ , suggesting efficient inward pebble drift and/or the formation of planetesimals.

</div>

<div id="div_fig1">

<img src="" alt="Fig11.1" width="50%"/><img src="" alt="Fig11.2" width="50%"/>

**Figure 11. -** **Top panels**: $\xco$ and $\cyo$ 2-1 line luminosities $(i=0^{\circ})$ for models with a gas mass of $\mgas = 10^{-3} \mathrm{M}_{\odot}$. The same models are shown in each panel, with colors highlighting the effect of different disk parameters on the line luminosity (see Table \ref{tab: model fixed parameters}). **Bottom panels**: as above, but showing $\cyo$ 2-1 and $\nhp$ 3-2 line luminosities $(i=0^{\circ})$ for models with a gas mass of $\mgas = 10^{-3} \mathrm{M}_{\odot}$, $x_{\rm CO} = 10^{-5}$, and $\zeta_{\rm CR} = 10^{-18} \mathrm{s}^{-1}$. The bottom right panel instead shows models with $\zeta_{\rm CR} \in [10^{-19}, 10^{-18}, 10^{-17}] \mathrm{s}^{-1}$.  (*fig: effect parameters*)

</div>
<div id="div_fig2">

<img src="" alt="Fig15.1" width="33%"/><img src="" alt="Fig15.2" width="33%"/><img src="" alt="Fig15.3" width="33%"/>

**Figure 15. -**  From top to bottom, left panels show the gas disk masses, gas-to-dust mass ratios and bulk CO gas abundance for sources in Ophiuchus (gray pentagons), Lupus (brown circles), and Upper Sco (green diamonds), all set against their dust disk mass. The markers show the median of the posterior distribution and the vertical lines denote its 16$^{\rm th}$ and 84$^{\rm th}$ quantile (see Section \ref{sec: measuring gas masses with MCMC}).
    The horizontal gray dashed lines in the top panel denotes the minimum mass solar nebula (MMSN; \citealt{Hayashi1981}) and the mass of Jupiter. The diagonal dotted lines show a constant gas-to-dust mass ratio.
    The right panels show histograms of the gas disk masses, gas-to-dust mass ratios and bulk CO gas abundance, obtained by summing the posterior distributions of individual sources and normalizing the histogram so that the sum over it equals ten. (*fig: final masses*)

</div>
<div id="div_fig3">

<img src="" alt="Fig16.1" width="33%"/><img src="" alt="Fig16.2" width="33%"/><img src="" alt="Fig16.3" width="33%"/>

**Figure 16. -** **Top left:** The correlation between $\mgas$ and $\rgas$, the latter being obtained from the best fit Nuker profile of the $\co$ 2-1 emission (see \citealt{AGEPRO_XI_gas_disk_sizes} for details). Black circles and purple diamonds represent sources in Lupus and Upper Sco, respectively. Blue hexagons shows the three disks with HD-based gas masses, TW Hya, DM Tau, and GM Aur, denoted by "T", "D", and "G". The orange dashed line shows a powerlaw fit to the observations. Shown in gray are analytically calculated $\rgas$ for different combinations of $\mgas$ and $\rc$ using the analytical equation for $\rgas$ from \cite{Trapman2023}. Here we used $L_*=0.3 \mathrm{L}_{\odot}$, $\gamma=1$, and $\xi =0$(see \citealt{Trapman2023} for details).
    **Top right:**$\rdust$ versus $\mgas$ for the same sources. Note that the $\rdust$ shown here were measured from the intensity profiles fitted to the visibilities (see \citealt{AGEPRO_X_substructures} for details).
    **Bottom:** The correlation between $\mgas$ and $\rc$ analytically derived from $\mgas$ and $\rgas$ following the method from \cite{Trapman2023}(see Appendix \ref{app: Rc derivation} for details.)  (*fig: Mgas-Rco-Rc correlation*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.10738"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\BT}[1]{\textcolor{black}{#1}}$
$\newcommand{\PP}[1]{\textcolor{black}{#1}}$
$\newcommand{\BTbis}[1]{\textcolor{black}{#1}}$
$\newcommand{\revBT}[1]{\textcolor{black}{#1}}$</div>



<div id="title">

# The ALMA Survey of Gas Evolution of PROtoplanetary Disks (AGE-PRO):\ VII. Testing accretion mechanisms from disk population synthesis

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.10742-b31b1b.svg)](https://arxiv.org/abs/2506.10742)<mark>Appeared on: 2025-06-13</mark> -  _Accepted for publication in ApJ for the special issue of AGE-PRO_

</div>
<div id="authors">

B. Tabone, et al. -- incl., <mark>A. Somigliana</mark>, <mark>K. Schwarz</mark>

</div>
<div id="abstract">

**Abstract:** The architecture of planetary systems depends on the evolution of the disks in which they form. In this work, we develop a population synthesis approach to interpret the AGE-PRO measurements of disk gas mass and size considering two scenarios: turbulence-driven evolution with photoevaporative winds and MHD disk-wind-driven evolution. A systematic method is proposed to constrain the distribution of disk parameters from the disk fractions, accretion rates, disk gas masses, and CO gas sizes. We find that turbulence-driven accretion with initially compact disks ( $R_0 \simeq 5-20 $ au), low mass-loss rates, and relatively long viscous timescales ( $t_{\nu,0} \simeq 0.4-3 $ Myr or $\alpha_{SS} \simeq 2-4 \times 10^{-4}$ ) can reproduce the disk fraction and gas sizes. However, the distribution of apparent disk lifetime defined as the $M_D/\dot{M}_*$ ratio is severely overestimated by turbulence-driven models. On the other hand, MHD wind-driven accretion can reproduce the bulk properties of the disk populations from Ophiuchus to Upper Sco assuming compact disks with an initial magnetization of about $\beta \simeq 10^5$ ( $\alpha_{DW} \simeq 0.5-1 \times 10^{-3}$ ) and a magnetic field that declines with time. More studies are needed to confirm the low masses found by AGE-PRO, notably for compact disks that question turbulence-driven accretion. The constrained synthetic disk populations can now be used for realistic planet population models to interpret the properties of planetary systems on a statistical basis.

</div>

<div id="div_fig1">

<img src="tmp_2506.10742/./fit_summary_PEW_v6_rev.png" alt="Fig12" width="100%"/>

**Figure 12. -** Summary of the constraints on turbulence-driven disk evolution obtained from disk dispersal time and median accretion rate measured in Lupus (namely, after 2).
The fitted values of \medMO(panel a) and \medMW(panel b) parameters are plotted as a function of \medtnu for various values of \medRO. The calculated median \medRO and $<M_D>$ at the estimated age of Lupus (2 Myr) are shown in panels c) and d). Median CO gas size and disk mass estimated by AGE-PRO are represented by a grey-shaded area. None of the synthetic populations can simultaneously reproduce the low disk mass and the disk size. Focusing on $<R_{CO}>$, we selected three best-fit models outlined by black circles and boxes. (*fig:fig_stage_2-3_PEW*)

</div>
<div id="div_fig2">

<img src="tmp_2506.10742/./fit_flow_chart_v3.png" alt="Fig2" width="100%"/>

**Figure 2. -** Schematic view of the fitting stages for the turbulent (left) and the MHD disk-wind (right) models. This step-by-step approach allows us to build synthetic populations consistent with the observations in a rationalized approach.
 (*fig:scheme_fitting*)

</div>
<div id="div_fig3">

<img src="tmp_2506.10742/./best_fit_CBF_v3.png" alt="Fig7" width="100%"/>

**Figure 7. -** Cumulative distribution of the accretion rate, the disk mass, and the disk lifetime for the best fit MHD disk-wind (red) and turbulence-driven model (blue) versus the AGE-PRO sample focusing on 1-3 Myr disks (black). The MHD wind model reproduces well the distributions. The salient discrepancy between the turbulence-driven model and the observations lies in the $M_D/\dot{M}_*$ ratio. The modeled cumulative distributions are computed assuming a detection threshold of $M_D= 10^{-5} M_{\odot}$. (*fig:CDF_best_fit*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.10742"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

521  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
